In [17]:
import csv
import sys
import zipfile
import io
import email
import json
archive = zipfile.ZipFile('enron-email-dataset.zip', 'r')
csv.field_size_limit(sys.maxsize)

def getstuff(filename):
    with archive.open(filename, "r") as file:
        csvfile = io.TextIOWrapper(file)
        datareader = csv.reader(csvfile)
        count = 0
        for row in datareader:
            yield row

In [14]:
gen = getstuff('emails.csv')

In [15]:
def skip(gen, n):
    for i in range(n):
        next(gen)
    return gen

In [ ]:
import email
count = 0
with open('filtered2.csv', "w") as filteredFile:
    writer = csv.writer(filteredFile)
    writer.writerow(next(gen))
    for row in gen:
        if (count == 250):
            skip(gen, 25000)
            count = 0
        msg = email.message_from_string(row[1])
        payload = msg.get_payload()
        if payload.startswith("---------------------- Forwarded") : continue
        if check_payload(payload):
            writer.writerow(row)
            count = count + 1
count

In [36]:
import requests
import json
duckling_url = 'http://192.168.99.100:8000/parse'
headers = {'content-type': 'application/x-www-form-urlencoded'}
def check_payload(text):
    payload = {'text': text}
    r = requests.post(duckling_url, data = payload, headers=headers)
    return any(x['dim'] == 'time' for x in r.json())

In [6]:
check_payload('Phillip, Below is the issues & to do list as we go forward with documenting the ')

False

In [4]:
import gzip
def get(filename):
    with open(filename, "r") as csvfile:
        datareader = csv.reader(csvfile)
        for row in datareader:
            yield row

In [2]:
gen = get('filtered2.csv')

In [5]:
mails = [x for x  in gen if x != [] and '----- Forwarded' not in x[1]]

In [19]:
import random
random.shuffle(mails)

In [10]:
import email
with open('filtered.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    count = 0
    for row in mails:
        msg = email.message_from_string(row[1])
        if msg['subject']:
            low = msg['subject'].lower()
            if (low.startswith('re:') or low.startswith('fwd:') or low.startswith('fw:')): continue
        writer.writerow(row)
        count = count + 1
count

2442

In [44]:
import os
import email
def parse_meeting_folder():
    count = 0
    with open('meatings.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(['category', 'mail'])
        folders = ["./EnronMeetings-Minorthird/train/bunch-1",
                   "./EnronMeetings-Minorthird/train/bunch-2",
                   "./EnronMeetings-Minorthird/train/bunch-3",
                   "./EnronMeetings-Minorthird/test/bunch4"]
        for folder in folders:
            for filename in os.listdir(folder):
                with open(os.path.join(folder, filename), 'r') as myfile:
                    data=myfile.read()
                msg = email.message_from_string(data)
                payload = msg.get_payload()
                if check_payload(payload):
                    writer.writerow(['1',data])
                    count = count + 1
    print(count)

In [45]:
parse_meeting_folder()

342


In [8]:
def get_emails(filename):
    emails = []
    with open(filename, "r") as csvfile:
        csvreader = csv.reader(csvfile)        
        for row in csvreader:
            emails.append(row[1])
    return emails

In [9]:
filtered = get_emails('filtered.csv')

In [11]:
meetings = get_emails('meetings.csv')

In [13]:
import random
fil_sample = random.sample(filtered,50)
meet_sample = random.sample(meetings,50)

In [14]:
data = fil_sample + meet_sample
random.shuffle(data)

In [29]:
results = [{'email':x, 'meetings':[{'subj':'', 'time':'', 'loc':'', 'guests':''}]} for x in data]

In [30]:
with open('data.json', 'w') as f:
    f.write(json.dumps(results))

In [20]:
results[0]

{'email': 'Message-ID: <6276880.1072121653621.JavaMail.evans@thyme>\nDate: Tue, 23 Oct 2001 08:40:04 -0700 (PDT)\nFrom: m.hall@enron.com\nTo: m.hall@enron.com, peggy.hedstrom@enron.com, w..white@enron.com, \n\tleslie.reeves@enron.com, c..gossett@enron.com, \n\trobert.superty@enron.com, a..allen@enron.com, s..theriot@enron.com, \n\tevelyn.aucoin@enron.com\nSubject: Energy Operations Staff Meeting\nCc: brandee.jackson@enron.com, sonia.hennessy@enron.com, \n\taraceli.romero@enron.com, yolanda.pena@enron.com\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nBcc: brandee.jackson@enron.com, sonia.hennessy@enron.com, \n\taraceli.romero@enron.com, yolanda.pena@enron.com\n\nPlease plan to attend a Staff Meeting with Bob Hall...\n\nWednesday, October 24, 2001',
 'guests': '',
 'loc': '',
 'subj': '',
 'time': ''}

In [24]:
from dateutil.parser import parse
str(parse('October 24, 2001'))

'2001-10-24 00:00:00'

In [31]:
with open('data.csv','w', newline='') as f:
    writer = csv.writer(f)
    for m in data:
        writer.writerow([m])